# <center> **TEXT SUMMARIZATION** </center>
**Thông tin thành viên**
  1. Lê Tấn Lộc - 20521546
  2. Nguyễn Hồng Anh Thư - 20520313
  3. Trương Thị Thanh Thanh - 20520767
  4. Nguyễn Thị Ngọc Nga - 20521641

# Cài đặt các thư viện cần thiết  

## Tải thư viện đánh giá rogue của python

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
%cd /content/drive/MyDrive/CS221.N12.KHCL/Final_Project/source_code/rogue
!git clone https://github.com/pltrdy/rouge


/content/drive/MyDrive/CS221.N12.KHCL/Final_Project/source_code/rogue
Cloning into 'rouge'...
remote: Enumerating objects: 286, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 286 (delta 13), reused 21 (delta 6), pack-reused 247
Receiving objects: 100% (286/286), 81.19 KiB | 934.00 KiB/s, done.
Resolving deltas: 100% (130/130), done.


In [6]:
cd rouge


/content/drive/MyDrive/CS221.N12.KHCL/Final_Project/source_code/rogue/rouge


In [7]:
!python setup.py install


running install
running bdist_egg
running egg_info
creating rouge.egg-info
writing rouge.egg-info/PKG-INFO
writing dependency_links to rouge.egg-info/dependency_links.txt
writing entry points to rouge.egg-info/entry_points.txt
writing requirements to rouge.egg-info/requires.txt
writing top-level names to rouge.egg-info/top_level.txt
writing manifest file 'rouge.egg-info/SOURCES.txt'
reading manifest file 'rouge.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'rouge.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/bin
copying bin/__init__.py -> build/lib/bin
copying bin/rouge_cmd.py -> build/lib/bin
creating build/lib/rouge
copying rouge/__init__.py -> build/lib/rouge
copying rouge/rouge.py -> build/lib/rouge
copying rouge/rouge_score.py -> build/lib/rouge
creating build/bdist.linux-x86_64
creating build/

In [8]:
import rouge
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from nltk.tokenize import sent_tokenize
import torch 
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import os


In [9]:
from rouge import Rouge

In [10]:
cd /content/drive/MyDrive/CS221.N12.KHCL/Final_Project/source_code

/content/drive/MyDrive/CS221.N12.KHCL/Final_Project/source_code


# Một số samples dùng để test

In [11]:
text1 = '''Ngày 27/3 , Cơ_quan Cảnh_sát điều_tra Công_an TP. Hưng_Yên , tỉnh Hưng_Yên cho biết , đơn_vị vừa ra quyết_định khởi_tố vụ án , khởi_tố bị_can đối_với đối_tượng Mai_Văn_Thương ( SN 1989 , trú tại đội 11 , thôn An_Chiểu 1 , xã Liên_Phương , TP. Hưng_Yên ) để điều_tra về hành_vi trộm_cắp tài_sản .
Theo tài_liệu điều_tra của cơ_quan công_an , vào_khoảng 7h30 ngày 13/3 , lợi_dụng gia_đình ông Mai_Văn_Thịnh ( chú ruột đối_tượng Thương ) ở cạnh nhà đi vắng , đối_tượng này đã đạp gãy chấn_song cửa_sổ , đột_nhập vào nhà ông Thịnh trộm_cắp 121kg thóc mang bán cho người cùng thôn lấy 700.000 đ .
Không dừng lại , sau đó đối_tượng tiếp_tục quay lại lục_soát tủ nhà ông Thịnh trộm_cắp 8.500.000 đ tiền_mặt ( ông Thịnh để dưới đáy tủ ) , rồi dùng số tiền trên để đi mua ma_tuý về sử_dụng và tiêu_xài hết 6.080.000 đ .
Đến ngày 15/3 , đối_tượng Thương đã đến Cơ_quan điều_tra Công_an TP. Hưng_Yên tự_thú và khai nhận toàn_bộ hành_vi phạm_tội của mình , đồng_thời giao_nộp cho cơ_quan công_an 3.120.000 đ .
Hiện Công_an TP. Hưng_Yên đã thu_giữ toàn_bộ 121kg thóc đối_tượng đã trộm_cắp để trao_trả cho gia_đình ông Thịnh .
Được biết Thương là đối_tượng nghiện ma_tuý từ nhiều năm nay , đã có 1 tiền_án về tội Tàng_trữ trái_phép chất ma_tuý bị TAND tỉnh Hưng_Yên xử_phạt 2 năm 3 tháng tù_giam .
Ra tù năm 2016 , đối_tượng này tiếp_tục có hành_vi cố_ý gây thương_tích , bị Công_an TP. Hưng_Yên ra quyết_định xử_phạt 2,5 triệu đồng .
Vụ án đang được Công_an TP. Hưng_Yên hoàn_thiện hồ_sơ để xử_lý Mai_Văn_Thương theo quy_định của pháp_luật .'''

summ1 = '''Với bản_tính ham chơi , lười làm , có nhiều tiền_án tiền_sự , lại nghiện ma_tuý , Thương đã đột_nhập vào nhà chú ruột để trộm hơn 1 tạ thóc và hơn 8 triệu đồng mang đi tiêu_xài .'''

text2 = '''Theo báo Sài_Gòn Giải_Phóng , ngày 17/3 , Công_an quận Gò_Vấp , TPHCM đang điều_tra truy_xét vụ trộm xe ô_tô hiệu Ford_Ranger ở chung_cư Hà_Đô , phường 3 , quận Gò_Vấp .
Thông_tin trên báo Thanh_Tra , Trung_tá Nguyễn_Đăng_Sơn , Phó Đội_trưởng đội Tổng_hợp , Công_an quận Gò_Vấp cho biết , theo thông_tin ban_đầu , vào_khoảng 10h ngày 13/3/2019 , anh Mai_Minh_N. ( sinh năm 1975 , tạm_trú tại lô D chung_cư Hà_Đô ) khi xuống lấy xe tại bãi giữ xe chung_cư Hà_Đô thì phát_hiện chiếc ô_tô hiệu Ford_Ranger , biển kiểm_soát : 51C-980.xx bị mất .
Vụ_việc nhanh_chóng được trình_báo đến Công_an địa_phương .
Nhận tin báo , Công_an quận Gò_Vấp có_mặt khám_nghiệm hiện_trường , lấy lời khai , điều_tra truy_xét .
Qua trích xuất camera an_ninh , công_an xác_định lúc 21h26 ngày 12/3 , một thanh_niên ( chưa rõ lai_lịch ) đi_lại chiếc xe của anh N. gửi ở bãi xe .
Sau đó , thanh_niên này mở_cửa , nổ máy chiếc xe ô_tô trên và rời khỏi bãi giữ xe .
Hình_ảnh ghi lại đối_tượng thực_hiện vụ trộm_cắp khá mờ nên việc điều_tra truy_xét khó_khăn .
Báo Tri_Thức_Trẻ thông_tin thêm , anh N. cho biết , chiếc ô_tô bị mất anh mua cách đây khoảng một năm , trị_giá gần 1 tỷ đồng , xe sắp hết hạn bảo_hiểm .
Sáng ngày 13/3/2019 , khi anh xuống lấy xe để đi mua bảo_hiểm thì phát_hiện chiếc xe không còn ở bãi giữ xe .
Hiện vụ_việc đang được điều_tra làm rõ .'''

summ2 = '''Xuống nhà_xe chung_cư Hà_Đô lấy xe , một người_dân bàng_hoàng phát_hiện chiếc xế hộp tiền tỷ của mình sau một đêm đã không_cánh_mà_bay .'''

text3 = '''Ngày 25/5 , đồn Công_an khu_công_nghiệp Vsip_Hải_Phòng Công_an huyện Thuỷ_Nguyên cho biết , đơn_vị vừa phối_hợp với văn_phòng cơ_quan CSĐT ( PC 01 ) Công_an TP. Hải_Phòng bắt_giữ và bàn_giao 2 đối_tượng Cà_Văn_Mái và Cà_Văn_Quân ( cùng SN 1993 , đều trú tại bản Thín B , xã Mường_Thín , huyện Tuần_Giáo , tỉnh Điện_Biên ) .
Mái và Quân bị bắt_giữ theo quyết_định truy_nã số 04 và 05 ngày 22/9/2018 của Công_an huyện Tuần_Giáo , tỉnh Điện_Biên với tội_danh Cố_ý gây thương_tích .
Trước đó , qua công_tác trao_đổi thông_tin , PC01 Công_an TP. Hải_Phòng tiếp_nhận thông_tin truy_nã về 2 đối_tượng Cà_Văn_Mái và Cà_Văn_Quân .
Năm 2018 , Mái và Quân là 2 đối_tượng trong vụ án Cố_ý gây thương_tích bị Công_an huyện Tuần_Giáo , tỉnh Điện_Biên điều_tra , xử_lý .
Sau khi gây án , 2 tên này đã nhanh chân bỏ trốn khỏi địa_phương và nhiều khả_năng đang lẩn_trốn tại khu_công_nghiệp Vsip_Hải_Phòng dưới vỏ bọc là những công_nhân xây_dựng .
Để truy bắt thành_công các đối_tượng trốn nã , tổ công_tác đội 3 PC01 và đồn Công_an KCN Vsip đã phối_hợp chặt_chẽ , triển_khai đồng_bộ các biện_pháp nghiệp_vụ .
Qua công_tác rà_soát địa_bàn , trinh_sát phát_hiện một nhóm người dân_tộc_thiểu_số đang làm công_nhân tại một công_trường xây_dựng trong KCN Vsip_Hải_Phòng , trong đó có 2 đối_tượng nghi_vấn là Mái và Quân .
Để che_giấu thân_phận , 2 tên đã dùng “ chiêu ” thay tên đổi họ và khai_báo quê_quán ở các địa_phương khác .
Hàng ngày , dưới “ vỏ bọc ” là những công_nhân , Mái cùng Quân sáng đi làm , tối về nhà_trọ đồng_thời luôn tỏ ra cần_cù , chịu_khó và hạn_chế giao_tiếp với những người xung_quanh .
Dù vậy , sau khi bí_mật tiến_hành xác_minh , nhất_là “ dấu_hiệu ” giọng nói có phần lơ_lớ của đối_tượng , trinh_sát đã chắc_chắn xác_định : 2 công_nhân trên chính là 2 kẻ trốn truy_nã theo quyết_định truy_nã của Công_an huyện Tuần_Giáo , tỉnh Điện_Biên .
Đến 18h30 ngày 16/5 , các lực_lượng công_an phối_hợp đã bắt gọn Mái và Quân .
Tại cơ_quan công_an , sau một hồi quanh_co , 2 đối_tượng dưới vỏ bọc là những công_nhân chịu_khó đã phải cúi đầu thừa_nhận chính là Cà_Văn_Mái và Cà_Văn_Quân .
Đồn Công_an KCN Vsip và PC01 Công_an TP. Hải_Phòng sau đó đã bàn_giao hồ_sơ và 2 đối_tượng cho Công_an tỉnh bạn xử_lý theo quy_định .
'''
summ3 = '''Sau khi gây án , bị công_an truy_nã gắt_gao , Mái và Quân đã thay tên đổi họ , xuống Hải_Phòng làm công_nhân và tỏ ra chăm_chỉ , hiền_lành .
'''
text4 = '''Sáng 20/10 , phòng Cảnh sát điều tra tội phạm về ma túy , Công_an tỉnh Đắk_Lắk cho biết , đang tiếp_tục điều_tra , xử_lý 21 nam_nữ thanh_niên tụ_tập trong quán karaoke sử_dụng ma_tuý bị lực_lượng công_an phát_hiện .
Trước đó , vào khoảng 1h sáng 19/10 , tổ công tác của phòng Cảnh sát hình sự và phòng Cảnh sát cơ động , Công_an tỉnh Đắk Lắk tiến_hành kiểm tra quán karaoke GaLaXy ở 391 đường Hùng Vương , thị_xã Buôn_Hồ , tỉnh Đắk Lắk .
Tại đây , tổ công_tác phát_hiện có 5 phòng đang hát .
Trong đó , 4 phòng hát có 22 thanh_niên nam , nữ đang có biểu hiện phê ma túy , bật nhạc to và nhảy múa .
Thời_điểm kiểm_tra tại 4 phòng hát này , tổ công tác thu giữ nhiều tang vật như : Ma túy đá , ketamin dùng để hít , thuốc lắc và cỏ Mỹ .
Lúc này , tổ công_tác đã đưa các đối_tượng trên về bệnh viện Đa_khoa TP. Buôn_Ma Thuột , tỉnh Đắk_Lắk để kiểm_tra .
Kết_quả cho thấy , có 21 đối tượng gồm 6 nữ và 15 nam dương tính với chất ma túy .
Tại cơ_quan công_an , các đối tượng khai nhận đã mua các chất ma túy trên rồi rủ nhau đi hát karaoke để tổ chức sử dụng .
Hiện , vụ_việc đang được phòng Cảnh sát điều tra tội phạm về ma túy Công_an tỉnh Đắk Lắk tiếp_tục điều_tra , xử lý theo quy định của pháp luật .'''
summ4 = '''Khi ập vào kiểm_tra quán karaoke , công_an phát_hiện có hàng chục nam_nữ thanh_niên với biểu_hiện phê ma_tuý , nhảy_nhót , lắc_lư trong tiếng nhạc chát_chúa .'''

text5 = '''Ngày 26/10 , đại_diện Công_an quận Hải_Châu , TP. Đà_Nẵng cho biết , đang làm_việc với Võ_Đức_Thắng ( 30 tuổi ) và Nguyễn_Quỳnh_Thy ( 27 tuổi ) , ngụ tại địa_phương về hành_vi vượt đèn_đỏ rồi đạp ngã 2 cảnh_sát đang làm nhiệm_vụ .
Vào sáng cùng ngày , Thắng và Thy điều_khiển xe_máy hiệu Exciter trên đường Quang_Trung .
Khi đến đoạn giao với Nguyễn_Thị_Minh_Khai , quận Hải_Châu thì Thắng rú ga vượt đèn_đỏ .
Lúc này , tổ Cảnh_sát Trật_tự , thuộc đội Cảnh_sát giao_thông trật_tự cơ_động Công_an quận Hải_Châu , gồm 4 cán_bộ , chiến_sĩ chạy mô_tô làm_việc tại đây thấy vậy liền truy_đuổi .
Biết cơ_quan_chức_năng đuổi theo , Thắng không dừng xe mà rú ga với tốc_độ nhanh hơn .
Các chiến_sĩ đuổi kịp , áp sát chặn xe thì Thắng dùng chân đạp vào xe mô_tô khiến 2 chiến_sĩ công_an ngã .
2 chiến_sĩ còn lại tiếp_tục truy_đuổi .
Khi đến ngã tư Quang_Trung – Nguyễn_Chí_Thanh , đôi nam_nữ bị chặn lại .
Vụ_việc khiến 1 chiến_sĩ bị_thương ở cổ_tay , khuỷu tay , xe mô_tô chuyên_dụng bị hư_hỏng phần đầu .
Hiện , cơ_quan_chức_năng đang tiếp_tục điều_tra , làm rõ vụ_việc .'''

summ5 = '''Thắng chở Thy vượt đèn_đỏ , khi biết có chiến_sĩ công_an đuổi theo , Thắng rồ ga chạy nhanh . Khi các chiến_sĩ đuổi kịp , Thắng dùng chân đạp ngã xe khiến 1 chiến_sĩ bị_thương .'''

text6 = '''Ngày 25/3 , trao_đổi với PV báo Người Đưa_Tin , luật_sư Đặng_Thị_Vân_Thịnh – Văn_phòng luật_sư Kết_Nối , đồng_thời là người bảo_vệ quyền và lợi_ích hợp_pháp cho cháu bé 9 tuổi ở Chương_Mỹ bị xâm_hại cho_hay , mới_đây VKSND TP. Hà_Nội đã phê_chuẩn quyết_định chuyển tội_danh Dâm_ô với người dưới 16 tuổi do Công_an huyện Chương_Mỹ khởi_tố sang tội_danh Hiếp_dâm người dưới 16 tuổi đối_với bị_can Nguyễn_Trọng_Trình ( 31 tuổi , trú xã Hoà_Chính , huyện Chương_Mỹ ) .
Liên_quan đến quyết_định phê_chuẩn của VKS về việc thay_đổi quyết_định khởi_tố bị_can , thay_đổi tội_danh , luật_sư Thịnh cho_rằng việc_làm này là rất kịp_thời và phù_hợp với quy_định của pháp_luật ; góp_phần xoa_dịu những nỗi_đau mà bị hại và gia_đình bị hại đã phải trải qua ; cũng như củng_cố thêm niềm_tin của quần_chúng nhân_dân vào các cơ_quan tiến_hành tố_tụng , vào công_lý .
Đồng_thời qua đó cũng tạo sức răn_đe , cảnh_báo , là bài_học lớn cho những đối_tượng có tư_tưởng , hành_vi xâm_hại trẻ_em .
Là người bảo_vệ cho cháu bé , lại là một người phụ_nữ , luật_sư Thịnh rất thấu_hiểu những nỗi_đau mà cháu bé đang và có_thể sẽ phải gánh_chịu trong tương_lai .
Cháu bé ở độ tuổi tâm_sinh_lý đang phát_triển .
Hành_vi mất nhân_tính của bị_can đã gây những ám_ảnh khủng_khiếp cho cháu bé không_chỉ thời_điểm hiện_tại mà có_thể sẽ còn đeo_bám cháu đến những chuỗi ngày về sau .
Do_vậy , luật_sư Thịnh cũng rất đau_đáu , trăn_trở về trường_hợp của cháu bé , nếu cháu không kịp_thời được sự quan_tâm , động_viên , chia_sẻ , cảm_thông của gia_đình , bạn_bè và xã_hội thì sẽ dẫn đến những khủng_hoảng về tâm_lý cho cháu .
Đánh_giá về tội_danh của bị_can , luật_sư Thịnh cho rằng : Hành_vi của Trình dùng vũ_lực đối_với cháu ( có hậu_quả là cháu bị gãy răng , rạn tay .. ) ; lợi_dụng tình_trạng không_thể tự_vệ được của cháu ( cháu là trẻ_em , trong khi Trình là một thanh_niên to_khoẻ ) để giao_cấu hoặc thực_hiện hành_vi quan_hệ tình_dục khác ( hậu_quả làm cháu bị rách màng_trinh và thủng tầng sinh_môn ) như_vậy đã đầy_đủ yếu_tố để cấu_thành tội Hiếp_dâm người dưới 16 tuổi .
Theo thông_tin gia_đình nạn_nhân cung_cấp , thời_điểm cháu Q. bị xâm_hại mới được 9 tuổi , 5 tháng 20 ngày ( tức_là dưới 10 tuổi ) .
Chiếu theo quy_định tại Điều 142 Bộ luật_Hình_sự năm 2015 , sửa_đổi bổ_sung năm 2017 về tội Hiếp_dâm người dưới 16 tuổi thì Trình sẽ bị khởi_tố ở khoản 3 điểm c là " Phạm_tội đối_với người dưới 10 tuổi ” .
“ Đối_tượng phạm_tội thuộc trường_hợp này thì bị phạt tù 20 năm , tù chung_thân hoặc tử_hình ” , luật_sư Thịnh cho biết .'''

summ6 = '''Trong trường_hợp bị chuyển tội_danh từ Dâm_ô với người dưới 16 tuổi sang tội_danh Hiếp_dâm người dưới 16 tuổi , gã “ đồ_tể ” Nguyễn_Trọng_Trình có_thể đối_mặt với mức án cao nhất là tử_hình .'''


In [12]:

viet_in = '''Khởi_tố kẻ_trộm hơn 1 tạ thóc và hơn 8 triệu đồng của chú ruột để lấy tiền mua ma_tuý

Ngày 27/3 , Cơ_quan Cảnh_sát điều_tra Công_an TP. Hưng_Yên , tỉnh Hưng_Yên cho biết , đơn_vị vừa ra quyết_định khởi_tố vụ án , khởi_tố bị_can đối_với đối_tượng Mai_Văn_Thương ( SN 1989 , trú tại đội 11 , thôn An_Chiểu 1 , xã Liên_Phương , TP. Hưng_Yên ) để điều_tra về hành_vi trộm_cắp tài_sản .
Theo tài_liệu điều_tra của cơ_quan công_an , vào_khoảng 7h30 ngày 13/3 , lợi_dụng gia_đình ông Mai_Văn_Thịnh ( chú ruột đối_tượng Thương ) ở cạnh nhà đi vắng , đối_tượng này đã đạp gãy chấn_song cửa_sổ , đột_nhập vào nhà ông Thịnh trộm_cắp 121kg thóc mang bán cho người cùng thôn lấy 700.000 đ .
Không dừng lại , sau đó đối_tượng tiếp_tục quay lại lục_soát tủ nhà ông Thịnh trộm_cắp 8.500.000 đ tiền_mặt ( ông Thịnh để dưới đáy tủ ) , rồi dùng số tiền trên để đi mua ma_tuý về sử_dụng và tiêu_xài hết 6.080.000 đ .
Đến ngày 15/3 , đối_tượng Thương đã đến Cơ_quan điều_tra Công_an TP. Hưng_Yên tự_thú và khai nhận toàn_bộ hành_vi phạm_tội của mình , đồng_thời giao_nộp cho cơ_quan công_an 3.120.000 đ .
Hiện Công_an TP. Hưng_Yên đã thu_giữ toàn_bộ 121kg thóc đối_tượng đã trộm_cắp để trao_trả cho gia_đình ông Thịnh .
Được biết Thương là đối_tượng nghiện ma_tuý từ nhiều năm nay , đã có 1 tiền_án về tội Tàng_trữ trái_phép chất ma_tuý bị TAND tỉnh Hưng_Yên xử_phạt 2 năm 3 tháng tù_giam .
Ra tù năm 2016 , đối_tượng này tiếp_tục có hành_vi cố_ý gây thương_tích , bị Công_an TP. Hưng_Yên ra quyết_định xử_phạt 2,5 triệu đồng .
Vụ án đang được Công_an TP. Hưng_Yên hoàn_thiện hồ_sơ để xử_lý Mai_Văn_Thương theo quy_định của pháp_luật .

Đối_tượng Mai_Văn_Thương tại cơ_quan công_an .'''

eng_in = '''The young boy who was hunting rabbits in the forest was not sure whether it had been the woman’s last cry or the child’s first that alerted him. He turned suddenly, sensing the possible danger, his eyes searching for an animal that was so obviously in pain. He had never known any animal to scream in quite that way before. He edged towards the noise cautiously; the scream had now turned to a whine but it still did not sound like any animal he knew. He hoped it would be small enough to kill; at least that would make a change from rabbit for dinner. The young boy moved stealthily towards the river, where the strange noise came from, running from tree to tree, feeling the protection of the bark against his shoulder blades, something to touch. Never stay in the open, his father had taught him. When he reached the edge of the forest, he had a clear line of vision all the way down the valley to the river, and even then it took him some time to realise that the strange cry emanated from no ordinary animal. He continued to creel) towards the whining, but he was out in the open on his own now. Then suddenly he saw the woman, with her dress above her waist, her bare legs splayed wide apart. He had never seen a woman like that before. He ran quickly to her side and stared down at her belly, quite frightened to touch. There, lying between the woman’s legs, was the body of a small, damp, pink animal, attached by I’l something that looked like rope. The young hunter dropped his freshly skinned rabbits and collapsed on his knees beside the little creature. He gazed for a long, stunned moment and then turned his eyes towards the woman, immediately regretting the decision. She was already blue with cold; her tired twentythree-year-old face looked middle-aged to the boy; he did not need to be told that she was dead. He picked up the slippery little body – had you asked him why, and no one ever did, he would have told you that the tiny fingernails clawing the crumpled face had worried him – and then he became aware that mother and child were inseparable because of the slimy rope. He had watched the birth of a lamb a few days earlier and he tried to remember. Yes, that’s what the shepherd had done, but dare he, with a child? The whining had stopped and he sensed that a decision was now urgent. He unsheathed his knife, the one he had skinned the rabbits with, wiped it on his sleeve and hesitating only for a moment, cut the rope close to the child’s body. Blood flowed freely from the severed ends. Then what had the shepherd done when the lamb was born? He had tied a knot to stop the blood. Of course, of course; he pulled some grass out of the earth beside him and hastily tied a crude knot in the cord. Then he took the child in his arms. He rose slowly from his knees, leaving behind him three dead rabbits and a dead woman who had given birth to this child. Before finally turning his back on the mother, he put her legs together, and pulled her dress down over her knees. It seemed to be the right thing to do. ‘Holy God,’ he said aloud, the first thing he always said when he had done something very good or very bad. He wasn’t yet sure which this was.'''

viet_in_2 = '''Hồi còn nhỏ, ngây ngô nghĩ rằng, một mối quan hệ, nếu bị khoảng cách chia xa, thể nào cũng sẽ tàn phai. Thế nhưng nếu không xa nhau, thì tình cảm cũng sẽ còn mãi. Thế là hồi đó, chia tay bạn bè cấp 2, tôi buồn hết 2 năm cấp 3. Cảm giác của một đứa không có gì bám víu giữa thành phố xa lạ nhưng lại không chịu chấp nhận một ai xung quanh. Riết rồi khó chia sẻ đến tận bây giờ. Vốn tôi nghĩ, thật tình những vấn đề của bản thân, chẳng ai hiểu nổi. Một đứa trẻ 15 tuổi, bảo là vấn đề to lớn như phá sản, tang tóc,... thật không phải. Nhưng vấn đề có nhỏ không? Đáp án là không! Thay đổi môi trường sống thật ra chỉ là giọt nước tràn li cho rất nhiều những vấn đề khác nhau, khó có lối ra, chỉ toàn nút thắt. Có chăng chỉ là giống sợi chỉ đỏ, khâu mà có nút chỗ nào, thì đành cắt đi khâu lại ở nút đó. Cứ thế bản thân vừa muốn kết nối, nhưng cũng tạo cho mình một rào cản khó có thể xóa bỏ. Nói là chia sẻ, nhưng chỉ dừng lại ở mặt chữ, là chia sẻ câu chuyện, trên nền chất liệu phong cách ngôn ngữ sinh hoạt, chứ lời nói chẳng có tí cảm xúc nào. Riết thành quen, bây giờ lời nói buông ra, cũng chẳng còn biết làm thế nào cho đằm thắm thứ cảm xúc đang bồn chồn muốn bộc phát trong tim. 
Lớn lên một chút, thôi thì môi trương thay đổi nhiều, chí ít đã quen những biến thiên hàng ngày của cuộc sống. Sống mà không có sự thay đổi, thì còn gọi gì là cuộc sống. Mãi bình an vui vẻ, chỉ có trên thiên đàng, mãi đau khổ sợ hãi, chỉ ở dưới địa ngục. Ba tầng trên thế gian, chắc nhân thế là hỗn loạn nhất! Bao nhiêu hỉ nộ ái ố đều phải kinh qua thì mới là người. Thế nhưng có lẽ cũng vì thế mà tôi vỡ òa, thật ra con người ta ở cạnh nhau mà lòng vẫn xa nhau được đấy thôi! Cái gọi là "cô đơn trong chính mối quan hệ của mình" thì giữa người nào cũng có. Người ta kết nối về mặt địa lí thì dễ dàng, chứ kết nối về mặt tâm hồn, e khó! Những con người của hôm nay, chúng tôi đối mặt nhau, ngồi cạnh nhau mà ngỡ là "một vòng trái đất". Người kia nghĩ gì, tôi không biết, cũng dần cảm thấy không còn muốn biết. Thành ra hai bên tự xây tường thành bao bọc cảm xúc. Bây giờ mình ôm nhau một cái được không? Để chất dẫn truyền thần kinh sinh ra làm ta hưng phấn? Tôi lại một lần nữa e khó! 
Bất cứ mối quan hệ nào, tôi cho rằng đều giống như 2 người đang đi trên dây. Khi người kia muốn tiến đến thì người còn lại phải đứng im và ngược lại, thành ra cứ mãi chệch nhịp. Còn nếu cả 2 cùng di chuyển được, thì phải có một cái gọi là "ĐỒNG TÂM". Suy nghĩ và cảm xúc cùng đồng điệu, chứ không cả 2 ngã nhào ngay! 
Nói chung suy đi xét lại, bí quyết để duy trì mối quan hệ tốt là gì? Không có quy luật nào cả. Làm sao mà để tất cả các phân số trên đời có chung mẫu số được? Vô lí. Rồi một người tiến một người lùi, cũng không xong, nếu muốn gần nhau hơn, chỉ có là cùng nhau hiểu về đối phương. Cái hiểu này không chỉ hiểu đối phương ở hiện tại, mà còn hiểu ở quá khứ. Từ đó mà chia sẻ và thấu hiểu. '''

# Methods

## Sử dụng TF-IDF

Chúng ta sẽ sử dụng  kĩ thuật TF-IDF cho giai đoạn Content Selection cho bài toán này và thuật toán KMeans cho giai đoạn Sentence Realization và Infomation Ordering

In [13]:
class tfidfSummarizer():
    def __init__(self, tfidf, tfidf_matrix, kmeans=True):# khởi tạo TF_IDF
        self.tfidf_matrix = tfidf_matrix
        self.tfidf = tfidf
        self.kmeans = kmeans


    def summarize(self, text):
        sentence_score_list, sentence_list = score_article(text, self.tfidf, self.tfidf_matrix, 0)# đánh giá các câu bằng score
        if self.kmeans:
            summary = generate_summary_kmeans(sentence_score_list, sentence_list)# sử dụng kmean
        else:
            summary = generate_summary_max(sentence_score_list, sentence_list, alpha=1.15)# sử dụng score để sắp xếp
        return summary


def score_sentence(sentence, tfidf, tfidf_matrix, a_index_):
  # Calculate the sum tfidf score of words as sentence score
  # Take in a sentence (string), and the article's index in the corpus (for word lookup)

  sentence_score = 0
  sentence = sentence.replace(' ,', '').strip()# câu đưa vào chỉ còn các từ,đã loại bỏ các dấu phẩy

  # For each word in a sentence, look up its score from tfidf matrix, return 0 if lookup fails
  for i_word in sentence.split(' '):# trích xuất các từ từ câu
    #print(i_word)
    score_word=tfidf_matrix[a_index_, tfidf.vocabulary_.get(i_word.lower(), 10)]
    sentence_score += score_word                                              #điểm của một từ trong câu được tính như sau: weight(w_i)=1 nếu -2log(lamda(w_i))<10
  print(sentence_score)                                                                                                                           #=0  ngược lại
  return sentence_score


def score_article(article, tfidf, tfidf_matrix, a_index_):#hàm để tính điểm cho article
  # Generate a list of scores of each sentence of an article
  # Take in an article (list of sentences), and the article's index in the corpus (for word lookup)

  sentence_score_list = []
  sentence_list = article.split('.')

  # For each sentence in an article, call score_sentence and append output to sentence_score_list
  for i_sentence in sentence_list:
    sentence_score = score_sentence(i_sentence, tfidf, tfidf_matrix, a_index_)#tính điểm cho các câu
    sentence_score_list.append(sentence_score)

  return sentence_score_list, sentence_list#t


def generate_summary_max(sentence_score_list, sentence_list, alpha=1.15):
  # Generate a summary from sentences that have score alpha times larger than the article' sentence mean score
  # Take in a list of sentence scores, list of sentences, thresholding multiplier alpha
  alpha=2
  sentence_score_array = np.asarray(sentence_score_list)
  mean_score = np.mean(sentence_score_array)# điểm TB cho các câu
 
  sentence_list_arr = np.asarray(sentence_list, dtype=object)
  summary = sentence_list_arr[sentence_score_array > alpha*mean_score]#các câu được tính điểm là các câu cao hơn alpha* điểm trung bình
  
  summary = ' '.join(summary)
  return summary

def generate_summary_kmeans(sentence_score_list, sentence_list, k=3):
    sentence_score_list = np.expand_dims(np.array(sentence_score_list), axis=1)

    if len(sentence_list) < k:# nếu số câu nhỏ hơn k
        k = len(sentence_list)

    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(np.array(sentence_score_list))
    label_max_tfidf = kmeans.labels_[np.argmax(sentence_score_list)]
    idxes = np.where(kmeans.labels_ == label_max_tfidf)[0]
    
    summary = ''
    for idx in idxes:
        summary += sentence_list[idx]
    print("K mean")
    return summary

In [14]:
def compute_tfidf(data_path):
    article_list = []
    for file in os.listdir(data_path):
        #print(file)
        with open(f'{data_path}/{file}', 'r') as f:
            article = f.read().splitlines()
            article_list.append(article[4:-1])   # leave out titles, abstracts
            f.close()

    for i in range(len(article_list)):
        article_list[i] = list(filter(None, article_list[i]))   # Remove blank spaces
        article_list[i] = ' '.join(article_list[i])

    # Fit Tfidf
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(article_list)
    print(tfidf_matrix)
    return tfidf, tfidf_matrix

In [ ]:
!git clone 'https://github.com/ThanhChinhBK/vietnews'

Cloning into 'vietnews'...
remote: Enumerating objects: 143827, done.
remote: Total 143827 (delta 0), reused 0 (delta 0), pack-reused 143827
Receiving objects: 100% (143827/143827), 194.68 MiB | 8.91 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Updating files: 100% (150704/150704), done.


In [17]:
# RUN SCRIPT 
path = 'vietnews/data/train_tokenized'
tfidf, tfidf_matrix = compute_tfidf(path)# khởi tạo tfidf với ngữ liệu vietnew

  (0, 50838)	0.00990855000102933
  (0, 2693)	0.013099973722723203
  (0, 90331)	0.029591474509837873
  (0, 78563)	0.04684358018531269
  (0, 2530)	0.03394300571455074
  (0, 104279)	0.0263948158095037
  (0, 27983)	0.045050864683396474
  (0, 96246)	0.043719077306079455
  (0, 108685)	0.019150713560084538
  (0, 18358)	0.018065835573240548
  (0, 110319)	0.026351680048339026
  (0, 17414)	0.051315747294359594
  (0, 37768)	0.03631812180778648
  (0, 87774)	0.025390908436366735
  (0, 41677)	0.04071424539989956
  (0, 56324)	0.015123087434244196
  (0, 21498)	0.019209512020662872
  (0, 106251)	0.020184943634277187
  (0, 108075)	0.02888354109667154
  (0, 32530)	0.024502097132901985
  (0, 37223)	0.04684358018531269
  (0, 86143)	0.025373671063853586
  (0, 42456)	0.0379349273627859
  (0, 96753)	0.036447255594012254
  (0, 64840)	0.02236963079766992
  :	:
  (29374, 16848)	0.025556607615957565
  (29374, 12238)	0.0947880600126814
  (29374, 58701)	0.06926305932743791
  (29374, 65747)	0.032042759397890835
  (2

In [18]:
summarizer = tfidfSummarizer(tfidf, tfidf_matrix,kmeans=True)
summary1 = summarizer.summarize(text5)
print(summary1)


0.024727242253748158
0.2656187032703045
0.18656223814389333
0.05236641639867973
0.08859449839169273
0.07298062378506352
0.14220986605139607
0.040557269265307935
0.04005483607702662
0.006949792155350552
0.05638192569852239
0.0
K mean
 Đà_Nẵng cho biết , đang làm_việc với Võ_Đức_Thắng ( 30 tuổi ) và Nguyễn_Quỳnh_Thy ( 27 tuổi ) , ngụ tại địa_phương về hành_vi vượt đèn_đỏ rồi đạp ngã 2 cảnh_sát đang làm nhiệm_vụ 


In [ ]:
rouge = Rouge()
rouge.get_scores(summary1, summ5)

[{'rouge-1': {'r': 0.21428571428571427,
   'p': 0.20689655172413793,
   'f': 0.21052631079101272},
  'rouge-2': {'r': 0.06060606060606061, 'p': 0.0625, 'f': 0.06153845653964538},
  'rouge-l': {'r': 0.14285714285714285,
   'p': 0.13793103448275862,
   'f': 0.14035087219452158}}]

## Sử dụng Centroid_based

In [19]:
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 5.0 MB/s eta 0:00:00


In [20]:
!pip install flashtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9307 sha256=f9ad1ffe4d211572b44c4b03a6cb3343b965385b51f3e71d3e4c1ec210d08d58
  Stored in directory: /root/.cache/pip/wheels/8d/62/8b/71813348245ae1bcbae179193bbc72db819e8057e89298a6ac
Successfully built flashtext


In [21]:
import re
import string
import unidecode
import numpy as np
from nltk.tokenize import sent_tokenize as nltk_sent_tokenize
from nltk.tokenize import word_tokenize as nltk_word_tokenize
from nltk.corpus import stopwords
from scipy.spatial.distance import cosine
from gensim.summarization.textcleaner import split_sentences as gensim_sent_tokenize
import flashtext

#hàm similarity dùng để đo mức độ tương đồng của 2 câu trong đoạn văn  dựa trên consine similarity
def similarity(v1, v2):
    score = 0.0
    if np.count_nonzero(v1) != 0 and np.count_nonzero(v2) != 0:
        score = ((1 - cosine(v1, v2)) + 1) / 2 #score của 2 vector v1,v2 được tính bằng 1-cosine(v1,v2)
    return score


In [22]:

class BaseSummarizer:

    extra_stopwords = ["''", "``", "'s"]

    def __init__(self,
                 language='english',
                 preprocess_type='nltk',
                 stopwords_remove=True,
                 length_limit=10,
                 debug=False):
        self.language = language
        self.preprocess_type = preprocess_type
        self.stopwords_remove = stopwords_remove
        self.length_limit = length_limit
        self.debug = debug
        if stopwords_remove:
            stopword_remover = flashtext.KeywordProcessor()
            for stopword in stopwords.words(self.language):
                stopword_remover.add_keyword(stopword, '')
            self.stopword_remover = stopword_remover
        return

    def sent_tokenize(self, text):
        if self.preprocess_type == 'nltk':
            sents = nltk_sent_tokenize(text, self.language)# đối với văn bản tiếng anh ta sẽ sử dụng thư viện NLTK
        else:
            sents = gensim_sent_tokenize(text)# sử dụng gensim với tiếng việt
        sents_filtered = []
        for s in sents:
            if s[-1] != ':' and len(s) > self.length_limit:
                sents_filtered.append(s)
            # else:
            #   print("REMOVED!!!!" + s)
        return sents_filtered

    def preprocess_text_nltk(self, text):
        sentences = self.sent_tokenize(text)# chuyển đổi về dạng sent token
        sentences_cleaned = []
        for sent in sentences:
            if self.stopwords_remove:
                self.stopword_remover.replace_keywords(sent)#loại bỏ stopword
            words = nltk_word_tokenize(sent, self.language)
            words = [w for w in words if w not in string.punctuation]# loại bỏ icon
            words = [w for w in words if w not in self.extra_stopwords]# loại bỏ kí tự đặc biệt
            words = [w.lower() for w in words]# chuyển về kiểu chữ thường
            sentences_cleaned.append(" ".join(words))
        return sentences_cleaned

    def preprocess_text_regexp(self, text):
        sentences = self.sent_tokenize(text)
        sentences_cleaned = []
        for sent in sentences:
            sent_ascii = unidecode.unidecode(sent)
            cleaned_text = re.sub("[^a-zA-Z0-9]", " ", sent_ascii)
            if self.stopwords_remove:
                cleaned_text = self.stopword_remover.replace_keywords(cleaned_text)
            words = cleaned_text.lower().split()
            sentences_cleaned.append(" ".join(words))
        return sentences_cleaned

    def preprocess_text(self, text):
        if self.preprocess_type == 'nltk':
            return self.preprocess_text_nltk(text)
        else:
            return self.preprocess_text_regexp(text)

    def summarize(self, text, limit_type='word', limit=100):
        raise NotImplementedError("Abstract method")

In [23]:
class CentroidBOWSummarizer(BaseSummarizer):

    def __init__(self,
                 language='english',
                 preprocess_type='nltk',
                 stopwords_remove=True,
                 length_limit=20,
                 debug=False,
                 topic_threshold=0.7,
                 sim_threshold=0.95):
        super().__init__(language, preprocess_type, stopwords_remove, length_limit, debug)
        self.topic_threshold = topic_threshold
        self.sim_threshold = sim_threshold
        return

    def summarize(self, text, limit_type='word', limit=100):
        raw_sentences = self.sent_tokenize(text)# chuyển đổi đoạn text về dạng tokenize với mỗi câu
        clean_sentences = self.preprocess_text(text)# tiền xử  lý

        vectorizer = CountVectorizer()# chuyển đổi về CountVectorizer
        sent_word_matrix = vectorizer.fit_transform(clean_sentences)

        transformer = TfidfTransformer(norm=None, sublinear_tf=False, smooth_idf=False)#khởi tạo TFIDF transfrom
        tfidf = transformer.fit_transform(sent_word_matrix)# convert về tfidf vector
        tfidf = tfidf.toarray()# convert về array

        centroid_vector = tfidf.sum(0)# tính tổng
        centroid_vector = np.divide(centroid_vector, centroid_vector.max())
        for i in range(centroid_vector.shape[0]):
            if centroid_vector[i] <= self.topic_threshold:
                centroid_vector[i] = 0

        sentences_scores = []
        for i in range(tfidf.shape[0]):
            score = similarity(tfidf[i, :], centroid_vector)
            sentences_scores.append((i, raw_sentences[i], score, tfidf[i, :]))

        sentence_scores_sort = sorted(sentences_scores, key=lambda el: el[2], reverse=True)

        count = 0
        sentences_summary = []
        for s in sentence_scores_sort:
            if count > limit:
                break
            include_flag = True
            for ps in sentences_summary:
                sim = similarity(s[3], ps[3])
                # print(s[0], ps[0], sim)
                if sim > self.sim_threshold:
                    include_flag = False
            if include_flag:
                # print(s[0], s[1])
                sentences_summary.append(s)
                if limit_type == 'word':
                    count += len(s[1].split())
                else:
                    count += len(s[1])

        summary = "\n".join([s[1] for s in sentences_summary])
        return summary

In [24]:

import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
summarizer = CentroidBOWSummarizer()
sum=summarizer.summarize(text1, limit=20)
print(sum)


Theo tài_liệu điều_tra của cơ_quan công_an , vào_khoảng 7h30 ngày 13/3 , lợi_dụng gia_đình ông Mai_Văn_Thịnh ( chú ruột đối_tượng Thương ) ở cạnh nhà đi vắng , đối_tượng này đã đạp gãy chấn_song cửa_sổ , đột_nhập vào nhà ông Thịnh trộm_cắp 121kg thóc mang bán cho người cùng thôn lấy 700.000 đ .
Không dừng lại , sau đó đối_tượng tiếp_tục quay lại lục_soát tủ nhà ông Thịnh trộm_cắp 8.500.000 đ tiền_mặt ( ông Thịnh để dưới đáy tủ ) , rồi dùng số tiền trên để đi mua ma_tuý về sử_dụng và tiêu_xài hết 6.080.000 đ .
Đến ngày 15/3 , đối_tượng Thương đã đến Cơ_quan điều_tra Công_an TP.


In [ ]:
from rouge import Rouge
rouge = Rouge()
rouge.get_scores(sum, summ1)

[{'rouge-1': {'r': 0.5, 'p': 0.20481927710843373, 'f': 0.29059828647527214},
  'rouge-2': {'r': 0.10810810810810811,
   'p': 0.0380952380952381,
   'f': 0.05633802431561225},
  'rouge-l': {'r': 0.38235294117647056,
   'p': 0.1566265060240964,
   'f': 0.2222222180992038}}]